In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pickle as pkl
from pprint import pprint
import time

In [2]:
with open('PLmatches.csv', 'r') as open_file:
    df = pd.read_csv(open_file, index_col=0)

In [4]:
no_attend_df = df[df['match_attendance'].isna()]
tinydf = no_attend_df[:10]

In [10]:
tinydf.iloc[0,:]

hometeam_name                       Aston Villa
awayteam_name                         Liverpool
home_goals                                    1
away_goals                                    2
match_date                      Sat 11 Aug 2007
match_timestamp                   1186848900000
match_referee                        Mike Riley
match_stadium            Villa Park, Birmingham
match_attendance                            NaN
home_possession                            51.3
away_possession                            48.7
home_shotsontarget                            4
away_shotsontarget                            5
home_shots                                   12
away_shots                                   18
home_touches                                574
away_touches                                578
home_passes                                 367
away_passes                                 356
home_tackles                                 29
away_tackles                            

with open('team_aliases_dict.pkl', 'rb') as open_file:
    team_aliases = pkl.load(open_file)

pprint(team_aliases, indent=1)

with open('team_fullnames_dict.pkl', 'rb') as open_file:
    team_fullnames = pkl.load(open_file)
    
team_fullnames

In [23]:
base_wf_url = 'https://www.worldfootball.net/report/premier-league-'

In [20]:
def get_fullname(name):
    return team_fullnames[name]

In [29]:
def get_season(rowindex):
    if rowindex < 5947:
        return '2006-2007'
    elif rowindex < 6327:
        return '2007-2008'
    elif rowindex < 6707:
        return '2008-2009'
    else:
        return '2009-2010'

In [6]:
options = Options()

In [74]:
def get_attendance(row):
    '''Scrapes and returns an attendance value from worldfootball.net
    -- if attendance already exists, returns existing value
    '''
    working_wf_url = base_wf_url

    if not np.isnan(row['match_attendance']):
        return row['match_attendance']
    else:
        working_wf_url += get_season(row.name)
        working_wf_url += '-'
        
        home = get_fullname(row['hometeam_name']).lower()
        away = get_fullname(row['awayteam_name']).lower()
        
        working_wf_url += '-'.join(home.split(' '))
        working_wf_url += '-'
        working_wf_url += '-'.join(away.split(' '))

        driver = webdriver.Firefox(firefox_options=options)
        driver.get(working_wf_url)
        attend_str = driver.find_element_by_xpath('//img[@title="Attendance"]/../..').text #find the attendance image, go up 2 levels, then get text
        attend_int = int(attend_str.replace('.',''))
        
        driver.quit()
        
        return attend_int

In [44]:
tinydf['match_attendance'] = tinydf.apply(get_attendance, axis=1)

/home/dlahtou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [45]:
tinydf

,hometeam_name,awayteam_name,home_goals,away_goals,match_date,match_timestamp,match_referee,match_stadium,match_attendance,home_possession,...,away_fouls,home_leaguerank,away_leaguerank,home_matchesplayed,away_matchesplayed,home_goaldifferential,away_goaldifferential,home_leaguepts,away_leaguepts,matchweek
5947,Aston Villa,Liverpool,1,2,Sat 11 Aug 2007,1186848900000,Mike Riley,"Villa Park, Birmingham",42640,51.3,...,14,14,7,1,1,-1,1,0,3,Matchweek 1
5948,Bolton Wanderers,Newcastle United,1,3,Sat 11 Aug 2007,1186840800000,Chris Foy,"Macron Stadium, Bolton",25414,61.7,...,15,19,1,1,1,-2,2,0,3,Matchweek 1
5949,Derby County,Portsmouth,2,2,Sat 11 Aug 2007,1186840800000,Mike Dean,"Pride Park Stadium, Derby",32176,53.9,...,16,9,10,1,1,0,0,1,1,Matchweek 1
5950,Everton,Wigan Athletic,2,1,Sat 11 Aug 2007,1186840800000,Mark Clattenburg,"Goodison Park, Liverpool",39220,40.8,...,13,6,17,1,1,1,-1,3,0,Matchweek 1
5951,Middlesbrough,Blackburn Rovers,1,2,Sat 11 Aug 2007,1186840800000,Andre Marriner,"Riverside Stadium, Middlesbrough",25058,58.0,...,18,16,5,1,1,-1,1,0,3,Matchweek 1
5952,Sunderland,Tottenham Hotspur,1,0,Sat 11 Aug 2007,1186832700000,Alan Wiley,"Stadium of Light, Sunderland",43967,48.9,...,13,8,18,1,1,1,-1,3,0,Matchweek 1
5953,West Ham United,Manchester City,0,2,Sat 11 Aug 2007,1186840800000,Peter Walton,"Boleyn Ground, London",34921,60.9,...,13,20,2,1,1,-2,2,0,3,Matchweek 1
5954,Arsenal,Fulham,2,1,Sun 12 Aug 2007,1186916400000,Phil Dowd,"Emirates Stadium, London",60093,66.3,...,20,4,15,1,1,1,-1,3,0,Matchweek 1
5955,Chelsea,Birmingham City,3,2,Sun 12 Aug 2007,1186921800000,Steve Bennett,"Stamford Bridge, London",41590,58.2,...,13,3,13,1,1,1,-1,3,0,Matchweek 1
5956,Manchester United,Reading,0,0,Sun 12 Aug 2007,1186930800000,Rob Styles,"Old Trafford, Manchester",75655,73.2,...,16,11,12,1,1,0,0,1,1,Matchweek 1


In [85]:
samplestring = "Att: 22,999"
def attend_format(row):
    if isinstance(row['match_attendance'], str):
        return int(row['match_attendance'].split(' ')[1].replace(',', ''))
    else:
        return np.nan

In [95]:
df['match_attendance'] = df.apply(attend_format, axis=1)

In [73]:
np.isnan(df.loc[6000, 'match_attendance'])

True

In [87]:
df.head(5)

,hometeam_name,awayteam_name,home_goals,away_goals,match_date,match_timestamp,match_referee,match_stadium,match_attendance,home_possession,...,away_fouls,home_leaguerank,away_leaguerank,home_matchesplayed,away_matchesplayed,home_goaldifferential,away_goaldifferential,home_leaguepts,away_leaguepts,matchweek
5567,Arsenal,Aston Villa,1,1,Sat 19 Aug 2006,1155996000000,Graham Poll,"Emirates Stadium, London",60023.0,72.9,...,19,9,10,1,1,0,0,1,1,Matchweek 1
5568,Bolton Wanderers,Tottenham Hotspur,2,0,Sat 19 Aug 2006,1156004100000,Phil Dowd,"Macron Stadium, Bolton",22899.0,37.8,...,22,5,17,1,1,2,-2,3,0,Matchweek 1
5569,Everton,Watford,2,1,Sat 19 Aug 2006,1155996000000,Peter Walton,"Goodison Park, Liverpool",39691.0,47.0,...,15,7,14,1,1,1,-1,3,0,Matchweek 1
5570,Newcastle United,Wigan Athletic,2,1,Sat 19 Aug 2006,1155996000000,Martin Atkinson,"Sports Direct Arena, Newcastle",52227.0,55.3,...,19,8,15,1,1,1,-1,3,0,Matchweek 1
5571,Portsmouth,Blackburn Rovers,3,0,Sat 19 Aug 2006,1155996000000,Alan Wiley,"Fratton Park, Portsmouth",19502.0,44.3,...,17,3,18,1,1,3,-3,3,0,Matchweek 1


In [84]:
type(np.nan)

float

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1697 entries, 5567 to 7264
Data columns (total 42 columns):
hometeam_name            1697 non-null object
awayteam_name            1697 non-null object
home_goals               1697 non-null int64
away_goals               1697 non-null int64
match_date               1697 non-null object
match_timestamp          1697 non-null int64
match_referee            1697 non-null object
match_stadium            1697 non-null object
match_attendance         1152 non-null float64
home_possession          1697 non-null float64
away_possession          1697 non-null float64
home_shotsontarget       1697 non-null int64
away_shotsontarget       1697 non-null int64
home_shots               1697 non-null int64
away_shots               1697 non-null int64
home_touches             1697 non-null int64
away_touches             1697 non-null int64
home_passes              1697 non-null int64
away_passes              1697 non-null int64
home_tackles            

In [98]:
with open('PLmatches.csv', 'w') as open_file:
    df.to_csv(open_file)

In [104]:
df.reset_index(inplace=True)

In [106]:
df['index'].diff().sort_values()

1          1.0
3021       1.0
3022       1.0
3023       1.0
3024       1.0
3025       1.0
3026       1.0
3027       1.0
3028       1.0
3029       1.0
3030       1.0
3031       1.0
3032       1.0
3033       1.0
3034       1.0
3035       1.0
3036       1.0
3037       1.0
3051       1.0
3050       1.0
3049       1.0
3048       1.0
3047       1.0
3046       1.0
3020       1.0
3045       1.0
3043       1.0
3042       1.0
3041       1.0
3040       1.0
         ...  
1506       1.0
1507       1.0
1508       1.0
1526       1.0
5          1.0
1511       1.0
1525       1.0
1524       1.0
1523       1.0
1522       1.0
1521       1.0
1520       1.0
1519       1.0
1510       1.0
1518       1.0
1516       1.0
1515       1.0
1514       1.0
1513       1.0
1512       1.0
1517       1.0
4346       2.0
1066       2.0
1899      10.0
2270      18.0
2650     988.0
3790    1546.0
3410    2125.0
4170    7923.0
0          NaN
Name: index, Length: 4549, dtype: float64

In [107]:
df.head()

,index,hometeam_name,awayteam_name,home_goals,away_goals,match_date,match_timestamp,match_referee,match_stadium,match_attendance,...,away_fouls,home_leaguerank,away_leaguerank,home_matchesplayed,away_matchesplayed,home_goaldifferential,away_goaldifferential,home_leaguepts,away_leaguepts,matchweek
0,5567,Arsenal,Aston Villa,1,1,Sat 19 Aug 2006,1155996000000,Graham Poll,"Emirates Stadium, London",60023.0,...,19,9,10,1,1,0,0,1,1.0,Matchweek 1
1,5568,Bolton Wanderers,Tottenham Hotspur,2,0,Sat 19 Aug 2006,1156004100000,Phil Dowd,"Macron Stadium, Bolton",22899.0,...,22,5,17,1,1,2,-2,3,0.0,Matchweek 1
2,5569,Everton,Watford,2,1,Sat 19 Aug 2006,1155996000000,Peter Walton,"Goodison Park, Liverpool",39691.0,...,15,7,14,1,1,1,-1,3,0.0,Matchweek 1
3,5570,Newcastle United,Wigan Athletic,2,1,Sat 19 Aug 2006,1155996000000,Martin Atkinson,"Sports Direct Arena, Newcastle",52227.0,...,19,8,15,1,1,1,-1,3,0.0,Matchweek 1
4,5571,Portsmouth,Blackburn Rovers,3,0,Sat 19 Aug 2006,1155996000000,Alan Wiley,"Fratton Park, Portsmouth",19502.0,...,17,3,18,1,1,3,-3,3,0.0,Matchweek 1


In [3]:
stadium_names = sorted(df['match_stadium'].unique())

In [60]:
df = df.apply(reassign_stadium_name, axis=1)

In [45]:
def reassign_stadium_name(row):
    if row['match_stadium'] == "St James' Park, Newcastle":
        row['match_stadium'] = "St. James' Park, Newcastle"
    return row

In [10]:
stadium_names

['Amex Stadium, Falmer',
 'Anfield, Liverpool',
 'Bloomfield Road, Blackpool',
 'Boleyn Ground, London',
 'Bramall Lane, Sheffield',
 'Britannia Stadium, Stoke',
 'Cardiff City Stadium, Cardiff',
 'Carrow Road, Norwich',
 'Craven Cottage, London',
 'DW Stadium, Wigan',
 'Emirates Stadium, London',
 'Etihad Stadium, Manchester',
 'Ewood Park, Blackburn',
 'Fratton Park, Portsmouth',
 'Goldsands Stadium, Bournemouth',
 'Goodison Park, Liverpool',
 "John Smith's Stadium, Huddersfield",
 'KCOM Stadium, Hull',
 'King Power Stadium, Leicester',
 'Liberty Stadium, Swansea',
 'Loftus Road, London',
 'London Stadium, London',
 'Macron Stadium, Bolton',
 'Madejski Stadium, Reading',
 'Molineux Stadium, Wolverhampton',
 'Old Trafford, Manchester',
 'Pride Park Stadium, Derby',
 'Riverside Stadium, Middlesbrough',
 'Selhurst Park, London',
 'Sports Direct Arena, Newcastle',
 'St Andrews, Birmingham',
 "St James' Park, Newcastle",
 "St. James' Park, Newcastle",
 "St. Mary's Stadium, Southampton",
 

In [214]:
with open('PLmatches.csv', 'w') as open_file:
    df.to_csv(open_file)

In [7]:
driver = webdriver.Firefox(firefox_options=options)

In [8]:
google_base_url = 'https://www.google.com/search?btnI=I&q='

In [9]:
def make_stdm_srch_str(stadium_name):
    srch_str = google_base_url + '+'.join(stadium_name.split(' ')) + '+stadium+wikipedia+football'
    return srch_str

In [11]:
stadium_info_dict = dict()
for stadium in stadium_names:
    driver.get(make_stdm_srch_str(stadium))
    try:
        coords = driver.find_element_by_xpath('//a[@title="Geographic coordinate system"]/../..').text
    except NoSuchElementException:
        coords = None
    try:
        capacity = driver.find_element_by_xpath('//a[@title="Seating capacity"]/../../td').text
    except NoSuchElementException:
        capacity = None
    stadium_info_dict[stadium] = {'coords': coords,
                                    'capacity': capacity}
    time.sleep(5)
    print(stadium)
    print(stadium_info_dict[stadium])

Amex Stadium, Falmer
{'coords': 'Coordinates: 50°51′42.56″N 0°4′59.80″W', 'capacity': '30,750[1]'}
Anfield, Liverpool
{'coords': 'Coordinates 53°25′50.98″N 2°57′39.05″W\nCoordinates: 53°25′50.98″N 2°57′39.05″W', 'capacity': '54,074[1]'}
Bloomfield Road, Blackpool
{'coords': 'Coordinates 53°48′17″N 3°2′53″W', 'capacity': '17,338'}
Boleyn Ground, London
{'coords': None, 'capacity': None}
Bramall Lane, Sheffield
{'coords': 'Coordinates 53°22′13″N 1°28′15″W\nCoordinates: 53°22′13″N 1°28′15″W', 'capacity': '32,702[1]'}
Britannia Stadium, Stoke
{'coords': None, 'capacity': None}
Cardiff City Stadium, Cardiff
{'coords': 'Coordinates 51°28′22″N 3°12′11″W\nCoordinates: 51°28′22″N 3°12′11″W', 'capacity': '33,280[1]'}
Carrow Road, Norwich
{'coords': 'Coordinates 52°37′19.66″N 1°18′31.15″E\nCoordinates: 52°37′19.66″N 1°18′31.15″E', 'capacity': '27,244[1]'}
Craven Cottage, London
{'coords': 'Coordinates 51°28′30″N 0°13′18″W\nCoordinates: 51°28′30″N 0°13′18″W', 'capacity': '25,700[1][2] (increasing 

In [111]:
driver.find_element_by_xpath('//a[@title="Seating capacity"]/../../td').text

'30,089[1]'

In [117]:
stadium_info_dict

{'Amex Stadium, Falmer': {'coords': 'Coordinates: 50°51′42.56″N 0°4′59.80″W',
  'capacity': '30,750[1]'},
 'Anfield, Liverpool': {'coords': 'Coordinates 53°25′50.98″N 2°57′39.05″W\nCoordinates: 53°25′50.98″N 2°57′39.05″W',
  'capacity': '54,074[1]'}}

In [41]:
with open('stadium_info.pkl', 'wb') as open_file:
    pkl.dump(stadium_info_dict, open_file)

In [129]:
stadium_info_dict['Boleyn Ground, London']

{'coords': '51°31′55″N 0°2′22″E', 'capacity': 35016}

In [18]:
for stadium in stadium_info_dict.keys():
    if not isinstance(stadium_info_dict[stadium]['capacity'], str):
        continue
    if len(stadium_info_dict[stadium]['capacity']) == 6:
        #stadium_info_dict[stadium]['capacity'] = stadium_info_dict[stadium]['capacity'][:6]
        stadium_info_dict[stadium]['capacity'] = stadium_info_dict[stadium]['capacity'].replace(',', '')
        stadium_info_dict[stadium]['capacity'] = int(stadium_info_dict[stadium]['capacity'])

In [40]:
for stadium in stadium_info_dict.keys():
    if not stadium_info_dict[stadium]['capacity']:
        print(f"{stadium}: {stadium_info_dict[stadium]['capacity']}")

In [39]:
stadium_info_dict['Britannia Stadium, Stoke']['capacity'] = 27740

17

In [162]:
df['home_goaldifferential'] = df['home_goaldifferential'].apply(pd.to_numeric)

In [206]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4549 entries, 5567 to 22468
Data columns (total 42 columns):
hometeam_name            4549 non-null object
awayteam_name            4549 non-null object
home_goals               4549 non-null int64
away_goals               4549 non-null int64
match_date               4549 non-null object
match_timestamp          4549 non-null int64
match_referee            4549 non-null object
match_stadium            4549 non-null object
match_attendance         4003 non-null float64
home_possession          4549 non-null float64
away_possession          4549 non-null float64
home_shotsontarget       4549 non-null int64
away_shotsontarget       4549 non-null int64
home_shots               4549 non-null int64
away_shots               4549 non-null int64
home_touches             4549 non-null int64
away_touches             4549 non-null int64
home_passes              4549 non-null int64
away_passes              4549 non-null int64
home_tackles           

In [184]:
df[df['home_leaguepts'] == "Matchweek 13"]

,hometeam_name,awayteam_name,home_goals,away_goals,match_date,match_timestamp,match_referee,match_stadium,match_attendance,home_possession,...,away_fouls,home_leaguerank,away_leaguerank,home_matchesplayed,away_matchesplayed,home_goaldifferential,away_goaldifferential,home_leaguepts,away_leaguepts,matchweek
22468,Southampton,Everton,4,1,Sun 26 Nov 2017,1511703000000,Kevin Friend,"St. Mary's Stadium, Southampton",30461.0,67.2,...,16,13,13,-2,-15,16,12,Matchweek 13,NaN,NaN


In [207]:
df['away_leaguepts'] = df.away_leaguepts.astype(int)

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4549 entries, 5567 to 22468
Data columns (total 42 columns):
hometeam_name            4549 non-null object
awayteam_name            4549 non-null object
home_goals               4549 non-null int64
away_goals               4549 non-null int64
match_date               4549 non-null object
match_timestamp          4549 non-null int64
match_referee            4549 non-null object
match_stadium            4549 non-null object
match_attendance         4003 non-null float64
home_possession          4549 non-null float64
away_possession          4549 non-null float64
home_shotsontarget       4549 non-null int64
away_shotsontarget       4549 non-null int64
home_shots               4549 non-null int64
away_shots               4549 non-null int64
home_touches             4549 non-null int64
away_touches             4549 non-null int64
home_passes              4549 non-null int64
away_passes              4549 non-null int64
home_tackles           

In [215]:
df.reset_index(inplace=True)

In [216]:
df.head(4)

,index,hometeam_name,awayteam_name,home_goals,away_goals,match_date,match_timestamp,match_referee,match_stadium,match_attendance,...,away_fouls,home_leaguerank,away_leaguerank,home_matchesplayed,away_matchesplayed,home_goaldifferential,away_goaldifferential,home_leaguepts,away_leaguepts,matchweek
0,5567,Arsenal,Aston Villa,1,1,Sat 19 Aug 2006,1155996000000,Graham Poll,"Emirates Stadium, London",60023.0,...,19,9,10,1,1,0,0,1,1,Matchweek 1
1,5568,Bolton Wanderers,Tottenham Hotspur,2,0,Sat 19 Aug 2006,1156004100000,Phil Dowd,"Macron Stadium, Bolton",22899.0,...,22,5,17,1,1,2,-2,3,0,Matchweek 1
2,5569,Everton,Watford,2,1,Sat 19 Aug 2006,1155996000000,Peter Walton,"Goodison Park, Liverpool",39691.0,...,15,7,14,1,1,1,-1,3,0,Matchweek 1
3,5570,Newcastle United,Wigan Athletic,2,1,Sat 19 Aug 2006,1155996000000,Martin Atkinson,"Sports Direct Arena, Newcastle",52227.0,...,19,8,15,1,1,1,-1,3,0,Matchweek 1


In [219]:
df['index'].diff().sort_values(ascending=False)

4180    7923.0
3420    2125.0
3800    1546.0
2660     988.0
2280      18.0
4356       2.0
1066       2.0
1522       1.0
1521       1.0
1520       1.0
1512       1.0
1523       1.0
1518       1.0
1517       1.0
1516       1.0
1515       1.0
1514       1.0
1524       1.0
1513       1.0
1519       1.0
1510       1.0
1511       1.0
1526       1.0
1509       1.0
1508       1.0
1507       1.0
1506       1.0
1505       1.0
1504       1.0
1503       1.0
         ...  
3054       1.0
3053       1.0
3052       1.0
3051       1.0
3050       1.0
3049       1.0
3048       1.0
3047       1.0
3046       1.0
3045       1.0
3044       1.0
3043       1.0
3042       1.0
3034       1.0
3028       1.0
3029       1.0
3030       1.0
3031       1.0
3032       1.0
3033       1.0
3035       1.0
3041       1.0
3036       1.0
3037       1.0
3038       1.0
3039       1.0
3040       1.0
1          1.0
1909       0.0
0          NaN
Name: index, Length: 4559, dtype: float64